In [1]:
import pyaudio

audio = pyaudio.PyAudio()

# List all available devices
print("Available audio devices:")
for i in range(audio.get_device_count()):
    device_info = audio.get_device_info_by_index(i)
    is_input = device_info['maxInputChannels'] > 0
    is_output = device_info['maxOutputChannels'] > 0
    device_type = []
    if "MAYA" not in device_info['name']:
        continue
    if is_input:
        device_type.append("Input: " + str(device_info['maxInputChannels']))
    if is_output:
        device_type.append("Output: " + str(device_info['maxOutputChannels']))
    device_type_str = ", ".join(device_type)
    print(f"Index {i}: {device_info['name']} - {device_type_str}")


Available audio devices:
Index 3: MAYA44USB Ch12 (ESI Audio Devic - Input: 4
Index 4: MAYA44USB Ch34 (ESI Audio Devic - Input: 4
Index 5: MAYA44USB Multichannel 4 (ESI A - Input: 4
Index 10: MAYA44USB Ch34 (ESI Audio Devic - Output: 4
Index 12: MAYA44USB Ch12 (ESI Audio Devic - Output: 4
Index 16: MAYA44USB Ch12 (ESI Audio Device (WDM) - MAYA44USB) - Input: 4
Index 17: MAYA44USB Ch34 (ESI Audio Device (WDM) - MAYA44USB) - Input: 4
Index 18: MAYA44USB Multichannel 4 (ESI Audio Device (WDM) - MAYA44USB) - Input: 4
Index 22: Speakers (ESI Audio Device (WDM) - MAYA44USB) - Output: 4
Index 23: MAYA44USB Ch34 (ESI Audio Device (WDM) - MAYA44USB) - Output: 4
Index 25: MAYA44USB Ch12 (ESI Audio Device (WDM) - MAYA44USB) - Output: 4
Index 27: Speakers (ESI Audio Device (WDM) - MAYA44USB) - Output: 4
Index 28: MAYA44USB Ch34 (ESI Audio Device (WDM) - MAYA44USB) - Output: 2
Index 31: MAYA44USB Ch12 (ESI Audio Device (WDM) - MAYA44USB) - Output: 2
Index 33: MAYA44USB Ch12 (ESI Audio Device (WDM) -